In [199]:
import pymssql
connect=pymssql.connect('KG-XXGLB-HESY','sa','123456','testing')
if connect: 
    print('connect success!')
cursor=connect.cursor()

connect success!


In [200]:
cursor.execute('DROP TABLE IF EXISTS receipt_records')
cursor.execute('DROP TABLE IF EXISTS leads_info')
cursor.execute('DROP TABLE IF EXISTS trading_records')
cursor.execute('DROP TABLE IF EXISTS staff_info')
cursor.execute('DROP TABLE IF EXISTS customer_info')

In [201]:
cursor.execute('''CREATE TABLE Staff_info(
    staff_id INT IDENTITY(1,1) PRIMARY KEY, 
    name VARCHAR(8) NOT NULL, 
    email VARCHAR(50), 
    tel CHAR(20), 
    supervisor CHAR(8))''')
cursor.execute('''CREATE TABLE Customer_info(
    customer_id INT IDENTITY(1,1) PRIMARY KEY, 
    customer_name CHAR(40) NOT NULL)''')
cursor.execute('''CREATE TABLE Leads_info(
    leads_id INT IDENTITY(1,1) PRIMARY KEY,
    title TEXT NOT NULL, 
    sop DATE, 
    state CHAR(10), 
    level CHAR(10), 
    defination TEXT, 
    stage CHAR(20), 
    application TEXT, 
    anual_demand FLOAT, 
    anual_sale FLOAT, 
    challenge CHAR(5), 
    progress TEXT,
    staff_id INT FOREIGN KEY REFERENCES Staff_info(staff_id), 
    customer_id INT FOREIGN KEY REFERENCES Customer_info(customer_id))''')
cursor.execute('''CREATE TABLE Trading_records(
    trade_no CHAR(8) PRIMARY KEY, 
    date DATE, 
    target_sale FLOAT, 
    sale_amount FLOAT, 
    target_benefits FLOAT, 
    benefits FLOAT,
    staff_id INT FOREIGN KEY REFERENCES Staff_info(staff_id),
    customer_id INT FOREIGN KEY REFERENCES Customer_info(customer_id))''')
cursor.execute('''CREATE TABLE Receipt_records(
    receipt_no CHAR(8) NOT NULL PRIMARY KEY,
    date DATE, 
    target_receipt FLOAT,
    receipt FLOAT,
    trade_no CHAR(8) FOREIGN KEY REFERENCES Trading_records(trade_no))''')



In [206]:
connect.commit()
connect.close()

In [202]:
from faker import Faker
fake=Faker(locale='zh_CN')

for _ in range(10):
    cursor.execute('''
                   INSERT INTO Staff_info(
                       name,
                       email,
                       tel,
                       supervisor) 
                   VALUES ('%s','%s','%s','%s')'''
                   %(fake.name(),
                     fake.ascii_company_email(),
                     fake.phone_number(),
                     fake.name()))

for _ in range(30):
    cursor.execute('''
                   INSERT INTO Customer_info(
                       customer_name)
                   VALUES ('%s')'''
                   %(fake.company()))


In [203]:
import radar
import numpy as np
import random
from random import randint
from faker import Faker
fake=Faker(locale='zh_CN')
state=['热','温','冷','立项']
level=['高','中','低']
stage=['阶段一','阶段二','阶段三']
challenge=['是','否']

for _ in range(50):
   cursor.execute('''
                  INSERT INTO Leads_info(
                     title, 
                     sop, 
                     state, 
                     level, 
                     defination, 
                     stage, 
                     application, 
                     anual_demand, 
                     anual_sale, 
                     challenge, 
                     progress,
                     staff_id,
                     customer_id) 
                  VALUES ('%s','%s','%s','%s','%s','%s','%s','%d','%d','%s','%s',%d,%d)'''
                  %(fake.word(), 
                    radar.random_date(), 
                    random.choice(state), 
                    random.choice(level), 
                    fake.sentence(nb_words=6, variable_nb_words=True, ext_word_list=None),
                    random.choice(stage),
                    fake.sentence(nb_words=6, variable_nb_words=True, ext_word_list=None),
                    randint(1000,5000),
                    randint(1000,5000),
                    random.choice(challenge),
                    fake.sentence(nb_words=6, variable_nb_words=True, ext_word_list=None),
                    randint(1,10),
                    randint(1,30)
                    )) 

In [204]:
for _ in range(100):
    cursor.execute('''
                   INSERT INTO Trading_records(
                       trade_no,
                       date,
                       target_sale,
                       sale_amount,
                       target_benefits,
                       benefits,
                       staff_id,
                       customer_id
                   ) VALUES ('%s','%s',%d,%d,%d,%d,%d,%d)
                   '''%(fake.ean8(),
                        # 条形码
                        radar.random_date(),
                        randint(4000,5000),
                        randint(1000,5000),
                        randint(1000,2000),
                        randint(100,1000),
                        randint(1,10),
                        randint(1,30)))

In [205]:
cursor.execute('SELECT trade_no FROM Trading_records')
trade_no=cursor.fetchall()

for _ in range(200):
    
    cursor.execute('''
                   INSERT INTO Receipt_records(
                       receipt_no,
                       date,
                       target_receipt,
                       receipt,
                       trade_no
                   ) VALUES ('%s','%s',%d,%d,'%s')
                   '''%(
                       fake.ean8(),
                       radar.random_date(),
                       randint(1000,5000),
                       randint(200,2000),
                       random.choice(trade_no)[0]
                   ))